<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TFs" data-toc-modified-id="TFs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TFs</a></span></li><li><span><a href="#Genes" data-toc-modified-id="Genes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Genes</a></span></li></ul></div>

# Search Output

This notebook generates json files for searching through ChIP-pro's genes and TFs.

In [1]:
import os
import pandas as pd

## TFs

In [2]:
prefix = '../ChiPdb/data/'
suffix = '/TF_list.csv'

organisms = [f.name for f in os.scandir(prefix)]
cols_of_interest = ['TF', 'Organism', 'Strain', 'Media', 'Supplement', 'accession', 'doi']

all_tfs = pd.DataFrame(columns = cols_of_interest + ['link'])

for o in organisms:
    try:
        tfs = pd.read_csv(prefix + o + suffix, index_col = 0)
    except NotADirectoryError:
        continue
        
    # build link
    for i in tfs.index:
        l = 'tf_dashboard.html?organism=' + o
        l += '&tf=' + tfs.TF[i]
        l += '&genome=' + tfs.genome_id[i]
        l += '&i=' + str(i)
        tfs.loc[i, 'link'] = l
    
    # take info of interest
    tfs = tfs[cols_of_interest + ['link']]
    
    # save
    all_tfs = all_tfs.append(tfs, ignore_index = True)
    
# get table to convert DOIs to PMIDs
# https://www.ncbi.nlm.nih.gov/pmc/pmctopmid/#converter
doi_pmid_df = pd.read_csv('doi_pmid_table.csv')
doi_pmid_df = doi_pmid_df.loc[doi_pmid_df.PMID.notna()]
doi2pmid = {'https://doi.org/'+d: p for d, p in zip(doi_pmid_df.DOI, doi_pmid_df.PMID.astype(int))}

# convert dois to pmids & clean up df
all_tfs['PMID'] = all_tfs.doi.replace(doi2pmid)
all_tfs = all_tfs[['TF', 'Organism', 'Strain', 'Media', 'Supplement', 'accession', 'PMID', 'link']]
all_tfs = all_tfs.rename({'accession':'Accession'}, axis = 1)

all_tfs

,TF,Organism,Strain,Media,Supplement,Accession,PMID,link
0,Lsr2,Streptomyces venezuelae,ATCC 10712,MS,--,GSE115439,31215866,tf_dashboard.html?organism=s_venezuelae&tf=Lsr...
1,LexA1,Leptospira interrogans,Copenhageni Fiocruz L1-130,EMJH,--,GSE73688,26762976,tf_dashboard.html?organism=l_interrogans&tf=Le...
2,BaeR,Escherichia coli,K-12 MG1655,LB,5% EtOH,GSE143856,33172971,tf_dashboard.html?organism=e_coli&tf=BaeR&geno...
3,CpxR,Escherichia coli,K-12 MG1655,LB,5% EtOH,GSE143856,33172971,tf_dashboard.html?organism=e_coli&tf=CpxR&geno...
4,Cra,Escherichia coli,K-12 MG1655,M9,"0.2% Fructose, Galactose, Acetate, or Glucose",GSE65643,29394395,tf_dashboard.html?organism=e_coli&tf=Cra&genom...
...,...,...,...,...,...,...,...,...
130,LexA1,Leptospira interrogans,Copenhageni Fiocruz L1-130,EMJH,--,GSE73688,26762976,tf_dashboard.html?organism=all_other&tf=LexA1&...
131,MksB,Corynebacterium glutamicum,ATCC 13032,BHI,--,PRJNA529385,32198399,tf_dashboard.html?organism=all_other&tf=MksB&g...
132,ParB,Corynebacterium glutamicum,ATCC 13032,BHI,--,PRJNA529385,32198399,tf_dashboard.html?organism=all_other&tf=ParB&g...
133,Smc,Corynebacterium glutamicum,ATCC 13032,BHI,--,PRJNA529385,32198399,tf_dashboard.html?organism=all_other&tf=Smc&ge...


In [3]:
all_tfs.to_json('../ChiPdb/data/tfs_search.json', orient='records')

## Genes

In [6]:
# get synonyms
gb = pd.read_csv('ecoli_GCF_000005845_2.gff', sep = '	', 
                 skiprows = 7, header = None, skipfooter = 1,
                 engine = 'python')
all_gene_info = pd.DataFrame(index = gb.index)
for i, row in gb.iterrows():
    elts = row[8].split(';')
    for e in elts:
        key_val = e.partition('=')
        all_gene_info.loc[i, key_val[0]] = key_val[2]

# rearrange into gene_info for synonym assignment
gene_info = all_gene_info[['locus_tag', 'gene_synonym', 'Name', 'gene', 'product']]
gene_info = gene_info.set_index('locus_tag')
gene_info = gene_info.loc[gene_info.gene_synonym.notna()]
gene_info = gene_info.loc[~gene_info.index.duplicated()]

In [9]:
prefix = '../ChiPdb/data/'

all_genes = pd.DataFrame(columns = ['Gene', 'Locus', 'Synonyms',
        'Organism', 'Strain', 'Binding_site_id',
        'Condition', 'Peak_start', 'Peak_end',
        'Peak_strength', 'link'])

for j, tf_row in all_tfs.iterrows():
    
    # get folder info from the link & read in file
    file = tf_row.link.replace('&', '=').split('=')
    f_dir = prefix + file[1] + '/' + file[5] + '/table/' 
    tables = [f.name for f in os.scandir(f_dir)]
    
    for t in tables:
        try:
            bt = pd.read_json(f_dir + t)
        except:
            continue
    
        for i, row in bt.iterrows():

            # get genes & loci
            genes = row.target_genes.split(',')
            loci = row.target_locus.split(',')

            # rearrange data for new table
            for g, l in zip(genes, loci):
                if g == '':
                    continue
                if (tf_row.Organism == 'Escherichia coli') and (tf_row.Strain == 'K-12 MG1655'):
                    try:
                        syn = gene_info.loc[l, 'gene_synonym']
                    except:
                        print('no synonyms for locus:', l)
                        syn = ''
                else:
                    syn = ''
                all_genes = all_genes.append({
                    'Gene':g,
                    'Locus':l,
                    'Synonyms': syn,
                    'Organism':tf_row.Organism,
                    'Strain':tf_row.Strain,
                    'Binding_site_id':row['index'],
                    'Condition':row.condition,
                    'Peak_start':row.binding_peak_start,
                    'Peak_end':row.binding_peak_end,
                    'Peak_strength':row.binding_peak_strength,
                    'link':tf_row.link}, ignore_index = True)
            
all_genes = all_genes.sort_values('Peak_start')

no synonyms for locus: b4659_1
no synonyms for locus: b4659_2
no synonyms for locus: b4580_1
no synonyms for locus: b4580_1
no synonyms for locus: b4580_1
no synonyms for locus: b0562_2
no synonyms for locus: b0562_1
no synonyms for locus: b4590
no synonyms for locus: b4570_1
no synonyms for locus: b4570_1
no synonyms for locus: b4493_2
no synonyms for locus: b4493_1
no synonyms for locus: b4493_2
no synonyms for locus: b4493_1
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_2
no synonyms for locus: b1459_2
no synonyms for locus: b1471_2
no synonyms for locus: b1471_2
no synonyms for locus: b1471_1
no synonyms for locus: b1471_2
no synonyms for locus: b1471_1
no synonyms for locus: b4696_2
no synonyms for locus: b4696_1
no synonyms for locus: b4696_2
no synonym

no synonyms for locus: b4496_1
no synonyms for locus: b4496_2
no synonyms for locus: b4496_1
no synonyms for locus: b4496_2
no synonyms for locus: b4496_1
no synonyms for locus: b4496_2
no synonyms for locus: b4496_1
no synonyms for locus: b4496_2
no synonyms for locus: b4496_1
no synonyms for locus: b4496_2
no synonyms for locus: b4571_1
no synonyms for locus: b4571_1
no synonyms for locus: b4571_1
no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synon

no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2641_2
no synonyms for locus: b2641_1
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_2
no synonyms for locus: b2850_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synon

no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2858_2
no synonyms for locus: b2891_2
no synonyms for locus: b2891_1
no synonyms for locus: b2891_2
no synonyms for locus: b2891_1
no synonyms for locus: b3046_2
no synonyms for locus: b3046_2
no synonyms for locus: b3046_2
no synonyms for locus: b3046_2
no synonyms for locus: b4569_1
no synonyms for locus: b4569_1
no synonyms for locus: b4569_1
no synonyms for locus: b4646_1
no synonyms for locus: b4646_2
no synonyms for locus: b4646_2
no synonyms for locus: b3423_2
no synonyms for locus: b3423_1
no synonyms for locus: b3443_1
no synonyms for locus: b3443_2
no synonyms for locus: b3443_1
no synonyms for locus: b3443_2
no synonyms for locus: b3443_1
no synonyms for locus: b3443_2
no synonyms for locus: b4660_2
no synonyms for locus: b4660_1
no synonyms for locus: b4660_2
no synonyms for locus: b4660_1
no synonyms for locus: b3504_2
no synon

no synonyms for locus: b4651_1
no synonyms for locus: b4651_2
no synonyms for locus: b4651_2
no synonyms for locus: b4615_1
no synonyms for locus: b4651_2
no synonyms for locus: b4615_1
no synonyms for locus: b4615_2
no synonyms for locus: b4615_1
no synonyms for locus: b4615_2
no synonyms for locus: b4615_1
no synonyms for locus: b4615_2
no synonyms for locus: b4615_1
no synonyms for locus: b4615_2
no synonyms for locus: b4652_1
no synonyms for locus: b4652_2
no synonyms for locus: b4615_2
no synonyms for locus: b4652_1
no synonyms for locus: b4652_2
no synonyms for locus: b4615_2
no synonyms for locus: b4652_1
no synonyms for locus: b4652_2
no synonyms for locus: b4615_2
no synonyms for locus: b4652_1
no synonyms for locus: b4652_2
no synonyms for locus: b4652_1
no synonyms for locus: b4652_2
no synonyms for locus: b4038_1
no synonyms for locus: b4038_2
no synonyms for locus: b4038_3
no synonyms for locus: b4038_1
no synonyms for locus: b4038_2
no synonyms for locus: b4038_3
no synon

...

no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_1
no synonyms for locus: b2850_2
no synonyms for locus: b2850_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2854_1
no synonyms for locus: b2854_2
no synonyms for locus: b2858_2
no synonyms for locus: b2858_2
no synonyms for locus: b2858_1
no synonyms for locus: b2858_2
no synonyms for locus: b2858_1
no synonyms for locus: b2858_2
no synonyms for locus: b2858_1
no synonyms for locus: b2858_2
no synonyms for locus: b2858_1
no synonyms for locus: b3046_2
no synonyms for locus: b3268_1
no synonyms for locus: b3268_2
no synonyms for locus: b3268_1
no synon

no synonyms for locus: b4493_2
no synonyms for locus: b4493_1
no synonyms for locus: b4493_2
no synonyms for locus: b4493_1
no synonyms for locus: b3423_2
no synonyms for locus: b3423_1
no synonyms for locus: b4488_1
no synonyms for locus: b4488_2
no synonyms for locus: b4488_1
no synonyms for locus: b4488_2
no synonyms for locus: b4488_1
no synonyms for locus: b4488_2
no synonyms for locus: b4488_1
no synonyms for locus: b4488_2
no synonyms for locus: b4488_2
no synonyms for locus: b4488_2
no synonyms for locus: b4654_2
no synonyms for locus: b4654_1
no synonyms for locus: b4654_2
no synonyms for locus: b4654_1
no synonyms for locus: b4580_1
no synonyms for locus: b4580_1
no synonyms for locus: b4580_1
no synonyms for locus: b0553_2
no synonyms for locus: b0553_1
no synonyms for locus: b0553_2
no synonyms for locus: b0553_1
no synonyms for locus: b0553_2
no synonyms for locus: b0553_1
no synonyms for locus: b0562_2
no synonyms for locus: b0562_1
no synonyms for locus: b0562_2
no synon

no synonyms for locus: b4654_2
no synonyms for locus: b4654_1
no synonyms for locus: b4580_1
no synonyms for locus: b4580_1
no synonyms for locus: b4580_1
no synonyms for locus: b0553_2
no synonyms for locus: b0553_1
no synonyms for locus: b0553_2
no synonyms for locus: b0553_1
no synonyms for locus: b0553_2
no synonyms for locus: b0553_1
no synonyms for locus: b0562_2
no synonyms for locus: b0562_1
no synonyms for locus: b4590
no synonyms for locus: b4491_1
no synonyms for locus: b4491_2
no synonyms for locus: b4491_1
no synonyms for locus: b4491_2
no synonyms for locus: b4491_1
no synonyms for locus: b4491_2
no synonyms for locus: b4521_1
no synonyms for locus: b4521_1
no synonyms for locus: b4492_1
no synonyms for locus: b4492_1
no synonyms for locus: b4492_1
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_1
no synonyms for locus: b1459_2
no synonyms for locus: b1459_1
no synonym

no synonyms for locus: b4038_1
no synonyms for locus: b4038_2
no synonyms for locus: b4038_3
no synonyms for locus: b4038_1
no synonyms for locus: b4038_2
no synonyms for locus: b4038_3
no synonyms for locus: b4038_3
no synonyms for locus: b4038_3
no synonyms for locus: b4038_3
no synonyms for locus: b4493_2
no synonyms for locus: b4493_1
no synonyms for locus: b2092_2
no synonyms for locus: b2092_1
no synonyms for locus: b2092_2
no synonyms for locus: b2092_1
no synonyms for locus: b2681_1
no synonyms for locus: b2681_2
no synonyms for locus: b4488_2
no synonyms for locus: b0017
no synonyms for locus: b1425
no synonyms for locus: b1437
no synonyms for locus: b2084
no synonyms for locus: b2596
no synonyms for locus: b4805
no synonyms for locus: b3004
no synonyms for locus: b4247
no synonyms for locus: b4250
no synonyms for locus: b0017
no synonyms for locus: b1425
no synonyms for locus: b1437
no synonyms for locus: b2084
no synonyms for locus: b2653
no synonyms for locus: b3007
no syno

KeyboardInterrupt: 

In [10]:
all_genes

,Gene,Locus,Synonyms,Organism,Strain,Binding_site_id,Condition,Peak_start,Peak_end,Peak_strength,link
0,SVEN_RS00030,SVEN_RS00030,,Streptomyces venezuelae,ATCC 10712,Lsr2-MS-1,lsr2 + MS,3811,4024,-1.880621,tf_dashboard.html?organism=s_venezuelae&tf=Lsr...
1,SVEN_RS01020,SVEN_RS01020,,Streptomyces venezuelae,ATCC 10712,Lsr2-MS-2,lsr2 + MS,228117,228485,-1.782210,tf_dashboard.html?organism=s_venezuelae&tf=Lsr...
2,SVEN_RS01220,SVEN_RS01220,,Streptomyces venezuelae,ATCC 10712,Lsr2-MS-3,lsr2 + MS,268339,269459,-1.243085,tf_dashboard.html?organism=s_venezuelae&tf=Lsr...
3,SVEN_RS01230,SVEN_RS01230,,Streptomyces venezuelae,ATCC 10712,Lsr2-MS-4,lsr2 + MS,269662,269930,-1.308862,tf_dashboard.html?organism=s_venezuelae&tf=Lsr...
4,SVEN_RS02195,SVEN_RS02195,,Streptomyces venezuelae,ATCC 10712,Lsr2-MS-5,lsr2 + MS,501613,502566,-1.668116,tf_dashboard.html?organism=s_venezuelae&tf=Lsr...
...,...,...,...,...,...,...,...,...,...,...,...
188764,coaD,b3634,"ECK3624,kdtB,yicA",Escherichia coli,K-12 MG1655,RpoB-fe-1579,rpob + fe,3808062,3808111,1.480000,tf_dashboard.html?organism=e_coli&tf=Cra&genom...
188765,waaU,b3623,"ECK3613,rfaK",Escherichia coli,K-12 MG1655,RpoB-fe-1579,rpob + fe,3808062,3808111,1.480000,tf_dashboard.html?organism=e_coli&tf=Cra&genom...
188766,waaZ,b3624,"ECK3614,rfaZ",Escherichia coli,K-12 MG1655,RpoB-fe-1579,rpob + fe,3808062,3808111,1.480000,tf_dashboard.html?organism=e_coli&tf=Cra&genom...
188767,waaY,b3625,"ECK3615,rfaY",Escherichia coli,K-12 MG1655,RpoB-fe-1579,rpob + fe,3808062,3808111,1.480000,tf_dashboard.html?organism=e_coli&tf=Cra&genom...


In [ ]:
all_genes.to_json('../ChiPdb/data/genes_search.json', orient='records')